# Introdução
___
Este arquivo tem por objetivo realizar o tratamento de dados do arquivo `cs_bisnode_panel.csv`.

### Objetivos do tratamento de dados:
* Remova as colunas ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D']  pois elas apresentam um percentual considerável de missing data ✅
* Remova de seus dados os registros do ano de 2016 ✅
* Criar uma coluna para variavel resposta (use o conceito de que uma empresa deixou de operar se ela esteve ativa no
ano X, mas não apresentou vendas em X + 2 anos) `# trabalhar nisso`
* Filtre para trabalhar apenas com empresas do ano de 2012 ✅
* usar np.where para ajustar Sales < 0 você já pode substituir por 0 ✅
* * Criar uma nova coluna para a escala logaritima de `Sales`✅
* Essa variável (`Sales`) é bastante assimétrica, concorda? Será que vale criar novas
colunas que representem o valor em log  dessa coluna?✅
* Será que isso também se aplica para as demais? `checar isso durante o loop de retreinamento`
* Crie novas colunas, como idade da empresa (faça isso pela subtração de
founded_year  e year ). Ah, cuide bem dos missing values. np.where pode ajudar
bastante!
* Filtre seus dados para ter empresas que possuem receita (revenue) abaixo de 10
milhões de euros e acima de 1000 euros
* Busque sempre embasar qualquer decisão de tratamento das variáveis. Faça isso
com o auxílio de estatísticas descritivas e também de gráficos de apoio.


# Importação dos dados e tratamento

In [76]:
import pandas as pd
import numpy as np

In [77]:
import missingno as msno

In [78]:
df = pd.read_csv("cs_bisnode_panel.csv")
df.head()

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1001034.0,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1001034.0,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1001034.0,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1001034.0,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1001034.0,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333


In [79]:
#msno.matrix(df)

## retirnado as colunas `'COGS', 'finished_prod', 'net_dom_sales','net_exp_sales', 'wages', 'D'` devido á alta taxa de missing
___

In [80]:
empresas_2012.columns

Index(['comp_id', 'begin', 'end', 'amort', 'curr_assets', 'curr_liab',
       'extra_exp', 'extra_inc', 'extra_profit_loss', 'fixed_assets',
       'inc_bef_tax', 'intang_assets', 'inventories', 'liq_assets',
       'material_exp', 'personnel_exp', 'profit_loss_year', 'sales',
       'share_eq', 'subscribed_cap', 'tang_assets', 'balsheet_flag',
       'balsheet_length', 'balsheet_notfullyear', 'year', 'founded_year',
       'exit_year', 'ceo_count', 'foreign', 'female', 'birth_year',
       'inoffice_days', 'gender', 'origin', 'nace_main', 'ind2', 'ind',
       'urban_m', 'region_m', 'founded_date', 'exit_date', 'labor_avg',
       'operates_within_2_years', 'sales_log', 'Idade_da_empresa'],
      dtype='object')

In [81]:
cols_to_rm = ['COGS', 'finished_prod', 'net_dom_sales','net_exp_sales', 'wages', 'D']

df = df.drop(columns = cols_to_rm)

In [82]:
df.columns

Index(['comp_id', 'begin', 'end', 'amort', 'curr_assets', 'curr_liab',
       'extra_exp', 'extra_inc', 'extra_profit_loss', 'fixed_assets',
       'inc_bef_tax', 'intang_assets', 'inventories', 'liq_assets',
       'material_exp', 'personnel_exp', 'profit_loss_year', 'sales',
       'share_eq', 'subscribed_cap', 'tang_assets', 'balsheet_flag',
       'balsheet_length', 'balsheet_notfullyear', 'year', 'founded_year',
       'exit_year', 'ceo_count', 'foreign', 'female', 'birth_year',
       'inoffice_days', 'gender', 'origin', 'nace_main', 'ind2', 'ind',
       'urban_m', 'region_m', 'founded_date', 'exit_date', 'labor_avg'],
      dtype='object')

## Removendo dados do ano de 2016
___

### convertendo as colunas para o formato datetime: 

In [83]:
df["begin"] = pd.to_datetime(df['begin'])
df["end"] = pd.to_datetime(df['end'])

In [84]:
# retinrando os dados em que o ano seja 2016 ou maior
df = df[df["begin"] < "2016"]
df.head()

,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,fixed_assets,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1001034.0,2005-01-01,2005-12-31,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,1229.629639,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1001034.0,2006-01-01,2006-12-31,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,725.925903,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1001034.0,2007-01-01,2007-12-31,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,1322.222168,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1001034.0,2008-01-01,2008-12-31,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,1022.222229,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1001034.0,2009-01-01,2009-12-31,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,814.814819,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333


### Criando a coluna indicadora de operação da empresa 
(se a empresa está operante em x+2 anos ou Não)

In [85]:
df_sorted = df.sort_values(by=['comp_id', 'begin'])

df['operates_within_2_years'] = df.groupby('comp_id')['sales'].shift(-2).apply(lambda x: 1 if x > 0 else 0 if pd.notna(x) else None)

df[["comp_id","begin","end","sales","operates_within_2_years"]].head(30) # para ver se o codigo funcionou


,comp_id,begin,end,sales,operates_within_2_years
0,1001034.0,2005-01-01,2005-12-31,6.275185e+04,1.0
1,1001034.0,2006-01-01,2006-12-31,6.462593e+04,1.0
2,1001034.0,2007-01-01,2007-12-31,6.510000e+04,1.0
3,1001034.0,2008-01-01,2008-12-31,7.808519e+04,1.0
4,1001034.0,2009-01-01,2009-12-31,4.538889e+04,0.0
5,1001034.0,2010-01-01,2010-12-31,9.929630e+03,0.0
6,1001034.0,2011-01-01,2011-12-31,0.000000e+00,0.0
7,1001034.0,2012-01-01,2012-12-31,0.000000e+00,0.0
8,1001034.0,2013-01-01,2013-12-31,0.000000e+00,0.0
9,1001034.0,2014-01-01,2014-12-31,0.000000e+00,NaN


* `groupby('comp_id')`: Agrupa os dados por empresa usando comp_id para que o cálculo de sales dois anos à frente seja feito separadamente para cada empresa.
* `.shift(-2)`: Pega o valor de sales duas linhas à frente dentro de cada grupo, o que equivale a dois anos à frente, devido à ordenação das datas.
* `apply(lambda x: 1 if x > 0 else 0)`: Define 1 se o valor for positivo e 0 caso contrário.

In [86]:
# Supondo que df seja o DataFrame que você forneceu
# Ordenar o DataFrame por 'comp_id' e 'begin'
df_sorted = df.sort_values(by=['comp_id', 'begin'])

# Criar um DataFrame temporário para facilitar as operações
df_temp = df_sorted.copy()
df_temp.set_index(['comp_id', 'begin'], inplace=True)

# Usar shift para criar colunas para os anos subsequentes
df_temp['sales_next_1'] = df_temp.groupby('comp_id')['sales'].shift(-1)
df_temp['sales_next_2'] = df_temp.groupby('comp_id')['sales'].shift(-2)

# Verificar a atividade com base nas vendas nos dois anos subsequentes
df_temp['operates_within_2_years'] = np.where(
    (df_temp['sales_next_1'] > 0) | (df_temp['sales_next_2'] > 0),
    1,
    np.where(df_temp['sales'].notna(), 0, np.nan)  # Se a linha é válida, mas não ativa, definir como 0
)

# Resetar o índice para retornar ao formato original
df_result = df_temp.reset_index()

# Mostrar as primeiras 30 linhas para verificação
print(df_result[["comp_id", "begin", "end", "sales", "operates_within_2_years"]].head(30))


      comp_id      begin        end         sales  operates_within_2_years
0   1001034.0 2005-01-01 2005-12-31  6.275185e+04                      1.0
1   1001034.0 2006-01-01 2006-12-31  6.462593e+04                      1.0
2   1001034.0 2007-01-01 2007-12-31  6.510000e+04                      1.0
3   1001034.0 2008-01-01 2008-12-31  7.808519e+04                      1.0
4   1001034.0 2009-01-01 2009-12-31  4.538889e+04                      1.0
5   1001034.0 2010-01-01 2010-12-31  9.929630e+03                      0.0
6   1001034.0 2011-01-01 2011-12-31  0.000000e+00                      0.0
7   1001034.0 2012-01-01 2012-12-31  0.000000e+00                      0.0
8   1001034.0 2013-01-01 2013-12-31  0.000000e+00                      0.0
9   1001034.0 2014-01-01 2014-12-31  0.000000e+00                      0.0
10  1001034.0 2015-01-01 2015-12-31  0.000000e+00                      0.0
11  1001541.0 2009-01-01 2009-12-31  9.259259e+02                      0.0
12  1001541.0 2010-01-01 

## Filtro para trabalhar apenas com empresas do ano de 2012
___

In [87]:
empresas_2012 = df[df['begin'] == "2012"]

In [88]:
empresas_2012.head()

,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,fixed_assets,...,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg,operates_within_2_years
7,1001034.0,2012-01-01,2012-12-31,140.740738,148.148148,21429.628906,0.0,0.0,0.0,340.740753,...,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333,0.0
14,1001541.0,2012-01-01,2012-12-31,481.481476,9629.629883,1303.703735,0.0,0.0,0.0,190566.671875,...,Domestic,5610.0,56.0,3.0,3,Central,2008-02-24,NaN,NaN,1.0
23,1002029.0,2012-01-01,2012-12-31,14929.629883,203885.187500,120444.453125,0.0,0.0,0.0,23459.259766,...,Domestic,2711.0,27.0,2.0,3,East,2006-07-03,NaN,0.458333,1.0
35,1003200.0,2012-01-01,2012-12-31,25.925926,22.222221,10996.295898,0.0,0.0,0.0,0.000000,...,Domestic,5630.0,56.0,3.0,1,Central,2003-10-21,2014-08-09,NaN,NaN
48,1007261.0,2012-01-01,2012-12-31,0.000000,255.555557,9207.407227,0.0,0.0,0.0,0.000000,...,Domestic,5610.0,56.0,3.0,1,Central,2010-08-26,2015-11-19,0.083333,NaN


## Trabalhando as incosistencias
___

### Ajustando a coluna sales

### Filtrando os dados para empresas com revenue abaixo de 10 milhões
Como não há uma coluna `revenue`, iremos considerar que `sales` corresponde à receita da empresa

In [89]:
empresas_2012['sales'].describe()

count    2.488600e+04
mean     4.164616e+05
std      3.269013e+06
min     -1.834445e+04
25%      4.975000e+03
50%      2.923519e+04
75%      1.078093e+05
max      1.058662e+08
Name: sales, dtype: float64

In [90]:
empresas_2012['sales'] = np.where(empresas_2012['sales'] < 0, 0, empresas_2012['sales']) # retirando valores negativos
empresas_2012 = empresas_2012[(df['sales'] > 1000) & (empresas_2012['sales'] < 10_000_000)] # filtrando os dados

C:\Users\helio\AppData\Local\Temp\ipykernel_25920\564670879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_2012['sales'] = np.where(empresas_2012['sales'] < 0, 0, empresas_2012['sales']) # retirando valores negativos
C:\Users\helio\AppData\Local\Temp\ipykernel_25920\564670879.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  empresas_2012 = empresas_2012[(df['sales'] > 1000) & (empresas_2012['sales'] < 10_000_000)] # filtrando os dados


In [91]:
empresas_2012['sales'].describe()

count    2.024000e+04
mean     2.381382e+05
std      7.735054e+05
min      1.003704e+03
25%      1.577037e+04
50%      4.515741e+04
75%      1.386509e+05
max      9.919548e+06
Name: sales, dtype: float64

### Checando a assimetria da coluna `Sales`

In [92]:
empresas_2012["sales"].skew()

7.21070047055794

Como a skewness esta acima de zero (bem acima) temos que a coluna `sales` possuí alta assímetria positiva

Criando uma nova coluna para a escala logaritima de `sales`

In [93]:
empresas_2012["sales_log"] = np.log1p(empresas_2012['sales'])
empresas_2012[["sales","sales_log"]].head()

,sales,sales_log
14,1.203704e+03,7.093989
23,1.136515e+06,13.943478
35,2.785185e+03,7.932429
56,4.336667e+05,12.980034
68,1.297296e+05,11.773216


In [94]:
empresas_2012["sales_log"].skew()

0.28412661200616257

Agora temos que a escala logaritima da coluna `sales` possui assimetria negativa, contudo está bem mais próxima de zero, indicando uma alta redução na assimetria

In [95]:
numeric_cols = empresas_2012.select_dtypes(include=[np.number])

skew_values = numeric_cols.skew()

print(skew_values.sort_values(ascending = False))

extra_exp                  141.373302
extra_inc                  129.229586
extra_profit_loss           69.127878
tang_assets                 49.023178
intang_assets               46.847875
curr_liab                   46.670760
fixed_assets                43.586954
inventories                 39.431871
subscribed_cap              35.054083
amort                       27.087272
liq_assets                  20.841177
share_eq                    17.386082
curr_assets                 15.375569
balsheet_notfullyear        14.050990
personnel_exp               12.146850
labor_avg                    9.219400
material_exp                 8.087935
sales                        7.210700
foreign                      2.795016
ceo_count                    2.683901
female                       1.110468
inoffice_days                1.062169
comp_id                      0.662560
sales_log                    0.284127
year                         0.000000
balsheet_flag                0.000000
urban_m     

### Criando a coluna `Idade_da_empresa`

In [96]:
empresas_2012['Idade_da_empresa'] = np.where(
    (empresas_2012['year'].isna()) | (empresas_2012['founded_year'].isna()),  # Condição: se 'year' ou 'founded_year' forem NaN
    np.nan,  # Se a condição for verdadeira (há missing values), preenche com NaN
    empresas_2012['year'] - empresas_2012['founded_year']  # Caso contrário, realiza a subtração normalmente
)
empresas_2012['Idade_da_empresa'].head()

14     4.0
23     6.0
35     9.0
56    20.0
68    11.0
Name: Idade_da_empresa, dtype: float64

## Criando o dataset para treinar o modelo

In [97]:
empresas_2012.to_csv("dados_para_o_R.csv", index = True)